In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.2 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,

)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              A virtual assistant answers questions and extracts information from a user based on the provided text which is a raw email that contains details about the job.
              USER: Text: {input_text}
              ASSISTANT: I’ve read this text.
              USER: What describes {entity_type} in the text?
              ASSISTANT:
           """

prompt = PromptTemplate(template=template, input_variables=["input_text","entity_type"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
text2 ="""
Remove/unsubscribe  |  Update your contact and subscribed mailing list(s)  |  Subscribe to mailing list(s) to receive requirements & resumes\r\n										\r\n								\r\n           						\r\n						\r\n                        \r\n    		              \r\n				       		\r\n								 \r\n									\r\n                                            From:\r\n		                                   Himesh,\r\n		                                   Largeton                                            \r\n									       himesh.srivastava@largeton.com\r\n									       Reply to:   himesh.srivastava@largeton.com\r\n	\r\n										\r\n								\r\n  						    \r\n						  \r\n                                                \r\n      						\r\n							\r\n								Job DescriptionJob Title: Senior Consultant 1-PEGA SSA- UI/UX Location: Trenton, NJ (Hybrid)  Duration: 1+ Year   CANDIDATE MUST BE IN NEW JERSEY     JOB DETAILS:   Required Skill: PEGA Senior Systems Architect Certification Required 5 Years PEGA UI/UX Required 5 Years Application design/Case design Required 5 Years PEGA Architect and how to integrate with customers technology tools Required 6 Years Developing solutions utilizing PEGA rules and procedures Required 4 Years Data Modeling Required 5 Years Automating Business Rules / Business Policies Required 5 Years Support of large-scale business centric Required 5 Years PEGA Administration and Security Required 6 Years  WebSphere or WebLogic Desired 6 Years MS/SQL or DB2 Desired 6 Years J2EE (JSP, Servlets, EJB, XML, Java) Desired 6 Years Related experience in ALL phases of Object Oriented software development, including design, configuration, testing, debugging, implementation. Desired 5 Years Ability to communicate clearly and effectively both written and oral with technical and non-technical personnel Required 4 Years 4-year college degree or equivalent technical experience Required 4 Years   Thanks And RegardsHimesh SrivastavaIT RecruiterContact:-5714639098E-mail:-himesh.srivastava@largeton.com  LARGETON INC. 13800 Coppermine Rd, Herndon, VA 20171.  Disclaimer: If you are not interested in receiving our e-mails. Then please reply with a ""REMOVE"" in the subject line. \r\n							\r\n        					\r\n      						\r\n      							\r\n      								 \r\n									\r\n										 Sign-Up for your account with PROHIRES POWERHOUSE Recruiting Portal to broadcast requirements & hotlists.
"""

In [ ]:
entity_type="Complete Company Address"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 ["13800 Coppermine Rd, Herndon, VA 20171"]


In [ ]:
entity_type="All Job Skills"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["UI/UX", "Application design/Case design", "PEGA Architect", "Data Modeling", "Automating Business Rules / Business Policies", "PEGA Administration and Security", "WebSphere", "WebLogic", "MS/SQL", "DB2", "J2EE", "Java"]


In [ ]:
entity_type="Recruiter Full Name"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["Himesh Srivastava"]


In [ ]:
entity_type="Recruiter Email Id"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["himesh.srivastava@largeton.com"]


In [ ]:
entity_type="Job Title"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["Senior Consultant 1-PEGA SSA- UI/UX"]


In [ ]:
entity_type="Location"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["Trenton", "NJ"]


In [ ]:
entity_type="Complete Job Description"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["Senior Consultant 1-PEGA SSA- UI/UX\n\nHybrid\n\n1+ Year\n\nCANDIDATE MUST BE IN NEW JERSEY\n\nUI/UX\n\nApplication design/Case design Required 5 Years\nPEGA Architect and how to integrate with customers technology tools Required 6 Years\nDeveloping solutions utilizing PEGA rules and procedures Required 4 Years\nData Modeling Required 


In [ ]:
entity_type="Company Full Name"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["LARGETON INC."]


In [ ]:
entity_type="Company Website"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

 ["www.largeton.com"]
